In [ ]:
from utils import *
import pandas as pd
import numpy as np
import arcpy
from arcgis.features import FeatureLayer

In [ ]:

service_url = 'https://maps.trpa.org/server/rest/services/Demographics/MapServer/27'

feature_layer = FeatureLayer(service_url)
tahoe_geometry_fields = ['YEAR', 'STATE', 'GEOGRAPHY', 'GEOID', 'TRPAID', 'NEIGHBORHOOD']
query_result = feature_layer.query(out_fields=",".join(tahoe_geometry_fields))
# Convert the query result to a list of dictionaries
feature_list = query_result.features

# Create a pandas DataFrame from the list of dictionaries
tahoe_geometry = pd.DataFrame([feature.attributes for feature in feature_list])

county_lookup = {
    '005': 'Douglas County (Tahoe Basin)',
    '017': 'El Dorado County (Tahoe Basin)',
    '031': 'Washoe County (Tahoe Basin)',
    '061': 'Placer County (Tahoe Basin)'
}

state_lookup = {
    '32': 'Nevada',
    '06': 'California'
}

inflation_adjustment = {
    '2000': 1.57,
    '2010': 1.24
}